In [23]:
import pandas as pd
import numpy as np
import torchaudio
from pathlib import Path
from fastai.vision.all import *
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from torchvision.utils import save_image

In [30]:
project_dir = Path("./birdclef-2022")
dataframe = pd.read_csv("./birdclef-2022/train_metadata.csv")

## data exploration

In [31]:
num_classes = len(dataframe.primary_label.unique())
dataframe.primary_label.unique()

array(['afrsil1', 'akekee', 'akepa1', 'akiapo', 'akikik', 'amewig',
       'aniani', 'apapan', 'arcter', 'barpet', 'bcnher', 'belkin1',
       'bkbplo', 'bknsti', 'bkwpet', 'blkfra', 'blknod', 'bongul',
       'brant', 'brnboo', 'brnnod', 'brnowl', 'brtcur', 'bubsan',
       'buffle', 'bulpet', 'burpar', 'buwtea', 'cacgoo1', 'calqua',
       'cangoo', 'canvas', 'caster1', 'categr', 'chbsan', 'chemun',
       'chukar', 'cintea', 'comgal1', 'commyn', 'compea', 'comsan',
       'comwax', 'coopet', 'crehon', 'dunlin', 'elepai', 'ercfra',
       'eurwig', 'fragul', 'gadwal', 'gamqua', 'glwgul', 'gnwtea',
       'golphe', 'grbher3', 'grefri', 'gresca', 'gryfra', 'gwfgoo',
       'hawama', 'hawcoo', 'hawcre', 'hawgoo', 'hawhaw', 'hawpet1',
       'hoomer', 'houfin', 'houspa', 'hudgod', 'iiwi', 'incter1',
       'jabwar', 'japqua', 'kalphe', 'kauama', 'laugul', 'layalb',
       'lcspet', 'leasan', 'leater1', 'lessca', 'lesyel', 'lobdow',
       'lotjae', 'madpet', 'magpet1', 'mallar3', 'masboo

**create spectogram files**  
https://www.kaggle.com/code/foolishboi/birdclef-convert-to-image-classification

In [32]:
items = get_files(project_dir, extensions='.ogg')

In [33]:
N_FFT = 2048
HOP_LEN = 1024

In [34]:
def create_spectrogram(filename):
    audio, sr = torchaudio.load(filename)
    specgram = torchaudio.transforms.MelSpectrogram(sample_rate=sr, 
                                                    n_fft=N_FFT, 
                                                    win_length=N_FFT, 
                                                    hop_length=HOP_LEN
                                                    ,
                                                    center=True,
                                                    pad_mode="reflect",
                                                    power=2.0,
                                                    norm='slaney',
                                                    onesided=True,
                                                    n_mels=224,
                                                    # mel_scale="htk"
                                                   )(audio).mean(axis=0)
    specgram = torchaudio.transforms.AmplitudeToDB()(specgram)
    specgram = specgram - specgram.min()
    specgram = specgram/specgram.max()
    
    
    return specgram

In [35]:
filename = items[1]
spec_default = create_spectrogram(filename)

In [36]:
filename.parent

Path('birdclef-2022/train_audio/sopsku1')

In [37]:
Path('spectograms/train_img').mkdir(parents=True, exist_ok=True)
Path('spectograms/test_img').mkdir(parents=True, exist_ok=True)

In [38]:
def create_image(filename):
    specgram = create_spectrogram(filename)
    if filename.parent.name in dataframe.primary_label.unique():
        Path('spectograms/train_img/{}'.format(filename.parent.name)).mkdir(parents=True, exist_ok=True)
        dest = Path('spectograms/train_img/{}'.format(filename.parent.name))/f'{filename.stem}.png'
    else:
         dest = Path('spectograms/test_img')/f'{filename.stem}.png'
    print(dest)
    save_image(specgram, dest)

In [40]:
_ = Parallel(n_jobs=-1)(delayed(create_image)(file) for file in tqdm(items))

  0%|          | 0/14853 [00:00<?, ?it/s]

/home/kilian/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'spectograms/test_img/soundscape_453028782.png'